<a href="https://colab.research.google.com/github/ritikkumar01111999/data_science/blob/main/Tag_prediction_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

import warnings

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss


warnings.filterwarnings("ignore")

# Import data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

# uploaded = files.upload()
data = pd.read_csv("/content/drive/MyDrive/tag_recommendation/data.csv")

Multi-label classification 

# Data cleaning

In [ ]:
# load stop words
# import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_word = stopwords.words('english')

In [ ]:

from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
def cleanig_data(dataset):
  from nltk.corpus import stopwords
  dataset.drop(columns=['Unnamed: 0'], inplace=True)
  dataset = dataset.dropna()
  # df['url'] = df['url'].str.lower()
  dataset['description'] = dataset['description'].str.lower()
  dataset['tags'] = dataset['tags'].str.lower()

  def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
  def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
  def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
  
  token=ToktokTokenizer()
  lemma=WordNetLemmatizer()

  def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))
  
  # dataset['description'] = dataset['description'].str.lower()
  dataset['description'] = dataset['description'].apply(cleanHtml)
  dataset['description'] = dataset['description'].apply(cleanPunc)
  dataset['description'] = dataset['description'].apply(keepAlpha)
  
  dataset['description'] = dataset['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
  dataset['description'] = dataset['description'].apply(lambda x: lemitizeWords(x))
  return dataset

In [ ]:
data_result = cleanig_data(data)

In [ ]:
data_result.head()

In [ ]:
data_result['description'][0]

**Tag Processing**

In [ ]:
def tag_processing(dataset):
  dataset['new_tags'] = dataset["tags"].apply(lambda x: x.split())
  all_tags = [item for sublist in dataset['new_tags'].values for item in sublist]
  print('TOtal number of tags are', len(all_tags))
  # getting unique set
  my_set = set(all_tags)
  unique_tags = list(my_set)
  print('Total number of unique are',len(unique_tags))
  return dataset

In [ ]:
cleaned_data = tag_processing(data_result)

****

**train - test Split**

In [ ]:
x_train=cleaned_data['description'][:3000]
x_test=cleaned_data['description'][3000:]

y_train = cleaned_data['tags'][:3000]
y_test =  cleaned_data['tags'][3000:]

**Encoding**

In [ ]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(), binary='true')

In [ ]:
multilabel_y_train = vectorizer.fit_transform(y_train)
multilabel_y_test = vectorizer.fit_transform(y_test)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
 print(multilabel_y_test.toarray())

In [ ]:
vectorizer = TfidfVectorizer(smooth_idf=True, norm="l2", \
                             tokenizer = lambda x: x.split(), sublinear_tf=False, ngram_range=(1,3))
x_train_multilabel = vectorizer.fit_transform(x_train)
x_test_multilabel = vectorizer.transform(x_test)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
print("Dimensions of train data X:",x_train_multilabel.shape, "Y :",multilabel_y_train.shape)
print("Dimensions of test data X:",x_test_multilabel.shape,"Y:",multilabel_y_test.shape)

**Training Model with logistic regression**

In [ ]:
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_multilabel,multilabel_y_train )
predictions = classifier.predict (x_test_multilabel)
predictions_train = classifier.predict (x_train_multilabel)

In [ ]:
print(predictions.toarray())

**Evaluating Model**

In [ ]:
y_true = multilabel_y_test.toarray()
y_logits = predictions.toarray()

In [ ]:
def result_check(x, y):
     count = 0
     for i in range(len(x)):
         if x[i] == y[i]:
             count += 1
     result = count / len(x)
     return result

In [ ]:
print(result_check(y_true[2],y_logits[2]))

In [ ]:
def subset_accuracy(y_true,y_predict,threshold):
  count = 0
  for j in range(len(y_true)):
    if result_check(y_true[j] , y_predict[j]) >= threshold:
      count = count + 1
  accuracy = count / len(y_true)
  return accuracy



**subset accuracy**

In [ ]:
threshold_list = [1,.9,.8,.7,.6,.5]
for i in threshold_list:
  print(f'for threshold {i} accuracy is = ',subset_accuracy(y_true,y_logits,threshold=i))

In [ ]:
print("train Accuracy :",metrics.accuracy_score(multilabel_y_train, predictions_train))
print("test Accuracy :",metrics.accuracy_score(multilabel_y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(multilabel_y_test,predictions))


precision = precision_score(multilabel_y_test, predictions, average='micro')
recall = recall_score(multilabel_y_test, predictions, average='micro')
f1 = f1_score(multilabel_y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(multilabel_y_test, predictions, average='macro')
recall = recall_score(multilabel_y_test, predictions, average='macro')
f1 = f1_score(multilabel_y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

In [ ]:
import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

In [ ]:
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')
multilabel_model = OneVsRestClassifier(xgb_estimator)
multilabel_model.fit(x_train_multilabel,multilabel_y_train)

# Preprocessing of text data


- bag of words
- tfidf vectorization
- word2vec
- own technique

Model with hyperparameter tuning
 - bag of words
 - tfidf vec.
 - word2vec
 - 


For every Model 
 - train accuracy in graph
 - test accuracy in graph

pretty table for all models


**Training Model**

In [ ]:
def avg_jacard(y_true,y_pred):
    '''
    see https://en.wikipedia.org/wiki/Multi-label_classification#Statistics_and_evaluation_metrics
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

def print_score_test(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")  
def print_score_train(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    print("Jacard score: {}".format(avg_jacard(y_train, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_train)*100))
    print("---") 

In [ ]:
dummy = DummyClassifier()
sgd = SGDClassifier()
lr = LogisticRegression()
mn = MultinomialNB()
svc = LinearSVC()
perceptron = Perceptron()
pac = PassiveAggressiveClassifier()
rfc = RandomForestClassifier()

for classifier in [dummy, sgd, lr, mn, svc, perceptron, pac , rfc]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_score_test(y_pred, classifier)
    print('accuracy_score:',accuracy_score(y_test,y_pred))

**Training model using  XGBOOSt**

In [ ]:
xgb_estimator = xgb.XGBClassifier(objective='binary:logistic')
multilabel_model = OneVsRestClassifier(xgb_estimator)
multilabel_model.fit(X_train, y_train)

In [ ]:
print('Accuracy on train data: {:.1f}%'.format(accuracy_score(y_train, multilabel_model.predict(X_train))*100))
print('Accuracy on test data: {:.1f}%'.format(accuracy_score(y_test, multilabel_model.predict(X_test))*100))

In [ ]:
print_score_test(multilabel_model.predict(X_test),multilabel_model )
print(classification_report(y_test, multilabel_model.predict(X_test)))

**hyperparameter tunig using SVC and kernel**





In [ ]:
model_to_set = OneVsRestClassifier(SVC(kernel="rbf"))

parameters = {
    "estimator__C": [1,2,3,4,5,6,7,8,9],
    "estimator__kernel": ["poly","rbf","linear"],
    "estimator__degree":[0,1, 2, 3, 4],
    
}

model_tunning = GridSearchCV(model_to_set, param_grid=parameters,
                             scoring='accuracy',cv = 3)

model_tunning.fit(X_cv, y_cv)

In [ ]:
model_tunning.best_score_

In [ ]:
ad=model_tunning.best_estimator_

In [ ]:
ad

In [ ]:
ad.fit(X_train, y_train)
y_pred = ad.predict(X_train)
print("Accuracy = ",accuracy_score(y_train,y_pred))

In [ ]:
y_pred = ad.predict(X_test)
print("Accuracy = ",accuracy_score(y_test,y_pred))


In [ ]:
print_score_test(y_pred, ad)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

**hyperparameter tuning using linearSVC**

In [ ]:
param_grid = {'estimator__C':[1,2,3,4,5,6,7,8,9] }

svc = OneVsRestClassifier(LinearSVC())
CV_svc = model_selection.GridSearchCV(estimator=svc, 
                                      param_grid=param_grid, cv= 5, scoring = 'accuracy')
CV_svc.fit(X_cv, y_cv)

In [ ]:
best_model = CV_svc.best_estimator_

best_model

In [ ]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_train)
print("Accuracy = ",accuracy_score(y_train,y_pred))

In [ ]:
y_pred = best_model.predict(X_test)
print("Accuracy = ",accuracy_score(y_test,y_pred))

In [ ]:
print_score_test(y_pred, best_model)

In [ ]:
print(classification_report(y_test,y_pred))

How to solve Multi - label classification Problem - https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff

Reason to use CV in model  -https://towardsdatascience.com/5-reasons-why-you-should-use-cross-validation-in-your-data-science-project-8163311a1e79

Nested cross validation  : https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

Reference

https://www.kaggle.com/miljan/predicting-tags-for-stackoverflow